In [2]:
import numpy as np
import matplotlib.pyplot as plt

from ann_flex import *
from metrics import *

In [3]:
import pandas as pd
import re
df = pd.read_csv("Data/RedditNews.csv")

In [4]:
def get_words(sentence):
    sentence=re.findall(r'[^b\'\"][^\"|^\'].*', sentence)
    return re.findall(r'[A-Za-z]+[\w]*|[\w^]*[A-Za-z]+[\w^\']*', sentence[0])

df['word_vec']=df.News.apply(get_words)

In [5]:
from gensim.models import Word2Vec

word2vec = Word2Vec(df.word_vec, min_count =5)
vocabulary=word2vec.wv.vocab
get_vect=word2vec.wv.get_vector

In [6]:
def add_vectors(words):
    vect=0
    count=0
    for word in words:
        if word in vocabulary:
            vect+=get_vect(word)
            count+=1
    if count>0: 
        return vect/count
    return np.zeros(100)

df['mean_vect']=df.word_vec.apply(add_vectors)

In [7]:
vects_by_date=pd.DataFrame(np.vstack(np.hstack((date,np.mean(np.vstack(df.loc[df.Date==date].mean_vect), axis=0))) for date in set(df.Date)))
vects_by_date.rename(columns={0:'Date'}, inplace=True)

In [8]:
dj=pd.read_csv('Data/DJIA_table.csv')
v=[]
for dates in vects_by_date.Date:
    v.append(dates in dj.Date.tolist())

d=[]
for dates in dj.Date:
    d.append(dates in vects_by_date.Date.tolist())

vbd=vects_by_date.iloc[v,:]
dj=dj.loc[d,:]


X=pd.DataFrame(np.column_stack((np.array(dj.sort_values(by='Date')),np.array(vbd.sort_values(by='Date')))))
X.drop([1,2,3,5,6,7],axis=1, inplace=True)

In [9]:
phi=np.array(X.iloc[:,1:])
phi[:,0]=np.log(phi[:,0].tolist())
n_back=3
for n_b in range(n_back): #get previous n_back days predictors for the current day's prediction
    phi=np.vstack((np.hstack((i,j)) for i,j in zip(phi,phi[1:,-101:])))


In [10]:
pd.DataFrame(phi).to_csv('phi.csv')

In [11]:
import numpy as np
import matplotlib.pyplot as plt

from ann_flex import *
from metrics import *
import pandas as pd

phi=np.array(pd.read_csv('phi.csv'))
n_total = phi.shape[0]
n_train = int(n_total*.6)
n_valid = int(n_total*.2)
n_test = int(n_total*.2)

phi=phi[np.random.permutation(n_total),:]

phi_train = phi[0:n_train,1:]
phi_valid = phi[n_train:n_train+n_valid,1:]
phi_test = phi[n_train+n_valid:n_train+n_valid+n_test,1:]

y_train = phi[0:n_train,0]
y_valid = phi[n_train:n_train+n_valid,0]
y_test = phi[n_train+n_valid:n_train+n_valid+n_test,0]

y_train=((y_train-y_train.mean())/50).reshape((n_train,1))
y_valid = ((y_valid-y_valid.mean())/50).reshape((n_valid,1))
y_test = ((y_test-y_test.mean())/50).reshape((n_test,1))


In [18]:
def fit_model(n_nodes, n_layers, mu, gamma, dropout, noise):
    nn = ANN([n_nodes]*n_layers, sigmoid)
    nn.fit(phi_train, y_train, eta = 1e-5, epochs=2000, mu=mu, gamma=gamma, dropout=dropout, noise=noise,Logistic=False)
    return OLS(y_valid, nn.predict(phi_valid))/n_valid


In [20]:
nodes=[30,34,38,42,46,50]
layers=[2,3,4]
mus=[.96,.965, .97, .975, .98]
gammas=[.91, .915, .92, .925, .93,.935, .94, .945, 95]
dropouts = [0,.05, .1, .15]
noises=[1e-10, 2e-10, 3e-10]
theta =[nodes,layers,mus,gammas, dropouts,noises]
theta_ind =[3,1,2,3,0,0]

best_J=fit_model(theta[0][theta_ind[0]],theta[1][theta_ind[1]],theta[2][theta_ind[2]],
                 theta[3][theta_ind[3]],theta[4][theta_ind[4]],theta[5][theta_ind[5]])
best_t=theta_ind
counter=0
while counter<100:
    counter+=1
    t=np.random.choice(len(theta))
    if theta_ind[t] < len(theta[t]):
        theta_ind[t]+=1
        fit=fit_model(theta[0][theta_ind[0]],theta[1][theta_ind[1]],theta[2][theta_ind[2]],
                 theta[3][theta_ind[3]],theta[4][theta_ind[4]],theta[5][theta_ind[5]])
        if fit < best_J:
            best_J=fit
            best_t=theta_ind
            print('count: {}, t {}, best Valid {}'.format(counter,t,  fit))
            continue
        theta_ind[t]-=1    
    if theta_ind[t] > 0:
        theta_ind[t]-=1
        fit=fit_model(theta[0][theta_ind[0]],theta[1][theta_ind[1]],theta[2][theta_ind[2]],
                 theta[3][theta_ind[3]],theta[4][theta_ind[4]],theta[5][theta_ind[5]])
        if fit < best_J:
            best_J=fit
            best_t=theta_ind
            print('count: {}, t {}, best Valid {}'.format(counter,t,  fit))
            continue
        theta_ind[t]+=1   
    

count: 1, t 3, best Valid 5.581773667262194
count: 8, t 3, best Valid 5.500353340272509
count: 14, t 0, best Valid 5.459188510880089
count: 18, t 1, best Valid 5.315550091201578
count: 20, t 2, best Valid 5.170324980945194


IndexError: list index out of range

In [32]:
def fit_model(n_nodes, n_layers, mu, gamma, dropout, noise):
    nn = ANN([n_nodes]*n_layers, sigmoid)
    nn.fit(phi_train, y_train, eta = 1e-5, epochs=5000, mu=mu, gamma=gamma, dropout=dropout, noise=noise,Logistic=False)
    return nn


In [33]:
nn=fit_model(theta[0][theta_ind[0]],theta[1][theta_ind[1]],theta[2][theta_ind[2]],
                 theta[3][theta_ind[3]],theta[4][theta_ind[4]],theta[5][theta_ind[5]])

In [34]:
print('R^2 validiate: {}'.format(r_squared(y_valid, nn.predict(phi_valid))))
print('R^2 test: {}'.format(r_squared(y_test, nn.predict(phi_test))))


R^2 validiate: 0.9544451946222219
R^2 test: 0.9440608667373217
